In [1]:
import os
def is_running_on_kaggle():
    return 'KAGGLE_URL_BASE' in os.environ

print('Running on Kaggle' if is_running_on_kaggle() else 'Running on local machine')

Running on local machine


In [2]:
DATA_PATH = '/kaggle/input/sentiment-analysis-dataset/' if is_running_on_kaggle() else 'data/'
TRAIN_FILE = DATA_PATH + 'train.csv'

In [3]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)


/home/andrewl73/anaconda3/envs/AML/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import pandas as pd
from datasets import Dataset

# Load the dataset from CSV
df = pd.read_csv(TRAIN_FILE)
df = df.rename(columns={'sentiment': 'label'})
df['label'] = df['label'].map({'positive': 1, 'negative': -1, 'neutral': 0})

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

# Apply the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 24732/24732 [00:06<00:00, 4072.90 examples/s]


In [5]:
from transformers import DataCollatorWithPadding

# Data collator to dynamically pad inputs to the maximum length of a batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Convert to PyTorch Dataset
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Assuming the dataset is split into train and test (if not, you'll need to split it)
train_size = int(0.8 * len(tokenized_datasets))
test_size = len(tokenized_datasets) - train_size

train_dataset, test_dataset = tokenized_datasets.train_test_split(test_size=test_size).values()


In [6]:
from transformers import GPT2ForSequenceClassification, Trainer, TrainingArguments

# Load the GPT-2 model for sequence classification
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=3)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
trainer.train()

  0%|          | 0/7422 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 1.95 GiB of which 22.62 MiB is free. Including non-PyTorch memory, this process has 1.93 GiB memory in use. Of the allocated memory 1.84 GiB is allocated by PyTorch, and 45.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 